In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
import keras
print(keras.__version__)

In [4]:
!gdown 1jIl8uXsrdRz3ZBxljjQNo0ssDT8LbJBx

In [5]:
!unzip will_not_travel_again_data.zip

In [6]:
train = pd.read_csv('data/train.csv', engine='pyarrow') # TODO
test = pd.read_csv('data/test.csv', engine='pyarrow') # TODO

test.drop(columns=[''], inplace=True)
train.head()

In [7]:
# TODO : Drop columns = ['user'] for both train and test sets
train.drop(columns=['user'], inplace=True)
test.drop(columns=['user'], inplace=True)

In [8]:
# TODO : check is_booking where checkIn_date or checkOut_date is null
missing_checkIn_checkOut = train[train['checkIn_date'].isnull() | train['checkOut_date'].isnull()]
percentage = (missing_checkIn_checkOut['is_booking'] == 0).mean()

In [9]:
# TODO : handle missing checkOut_date where checkIn_date values
if(percentage >= 0.99):
    train.dropna(subset=['checkIn_date', 'checkOut_date'], inplace=True)
else:
    train['checkIn_date'] = train['checkIn_date'].fillna(train['checkIn_date'].mode()[0])
    train['checkOut_date'] = train['checkOut_date'].fillna(train['checkOut_date'].mode()[0])

In [10]:
train.isna().mean()['destination_distance']

In [11]:
mean = train.groupby(['user_location_city', 'destination'])['destination_distance'].transform('mean') #TODO

train['destination_distance'].fillna(mean, inplace = True) #TODO
test['destination_distance'].fillna(mean, inplace = True) #TODO

In [12]:
train[train['destination_distance'] == 0]

In [13]:
train['destination_distance'].fillna(0, inplace=True) #TODO
test['destination_distance'].fillna(0, inplace=True) #TODO

In [14]:
train.isna().sum()

In [15]:
# TODO : change type of columns in time_columns into datetime64[ns]

time_columns = ['search_date', 'checkIn_date', 'checkOut_date']

for time_column in time_columns:
    train[time_column] = pd.to_datetime(train[time_column])
    test[time_column] = pd.to_datetime(test[time_column])

In [16]:
a = train[train['search_date'] > train['checkIn_date']].max()
print(a['checkIn_date'] - a['search_date'])

In [17]:
# TODO : Add Days of Stay

duration = train['checkOut_date'] - train['checkIn_date'] # TODO
train['duration'] = duration.dt.days # TODO
duration = test['checkOut_date'] - test['checkIn_date'] # TODO
test['duration'] = duration.dt.days # TODO

# TODO : Add Days between search_date and checkIn_date

days_between = train['checkIn_date'] - train['search_date'] # TODO
print(days_between)
train['days_between'] = days_between.dt.days # TODO
days_between = test['checkIn_date'] - test['search_date'] # TODO
test['days_between'] = days_between.dt.days # TODO

In [18]:
# TODO : Add Days of Stay

duration = train['checkOut_date'] - train['checkIn_date'] # TODO
train['duration'] = duration.dt.days # TODO
duration = test['checkOut_date'] - test['checkIn_date'] # TODO
test['duration'] = duration.dt.days # TODO

# TODO : Add Days between search_date and checkIn_date

days_between = train['checkIn_date'] - train['search_date'] # TODO
train['days_between'] = days_between.dt.days # TODO
days_between = test['checkIn_date'] - test['search_date'] # TODO
test['days_between'] = days_between.dt.days # TODO

In [19]:
train['search_date_hour'] = train['search_date'].dt.hour # TODO
train['search_date_dayofweek'] = train['search_date'].dt.dayofweek # TODO
train['checkIn_date_dayofweek'] = train['checkIn_date'].dt.dayofweek # TODO
train['search_date_month'] = train['search_date'].dt.month # TODO
train['checkIn_date_month'] = train['checkIn_date'].dt.month # TODO

test['search_date_hour'] = test['search_date'].dt.hour # TODO
test['search_date_dayofweek'] = test['search_date'].dt.dayofweek # TODO
test['checkIn_date_dayofweek'] = test['checkIn_date'].dt.dayofweek # TODO
test['search_date_month'] = test['search_date'].dt.month # TODO
test['checkIn_date_month'] = test['checkIn_date'].dt.month # TODO

In [20]:
is_booked = train[train['is_booking'] == 1] #TODO
not_booked = train[train['is_booking'] == 0] #TODO

In [21]:
import plotly.graph_objects as go

not_booked_percentages = not_booked['search_date_hour'].value_counts() / len(not_booked)
not_booked_percentages = [not_booked_percentages[i] for i in range(24)]

is_booked_percentages = is_booked['search_date_hour'].value_counts() / len(is_booked)
is_booked_percentages = [is_booked_percentages[i] for i in range(24)]

trace_not_booked = go.Bar(y = not_booked_percentages, name='Not Booked') # TODO
trace_is_booked = go.Bar(y = is_booked_percentages, name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Search Hour', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./search_hour.json')

In [22]:
# TODO: Make Bins for search_date_hour (Optional)
train['search_date_hour_bin'] = train['search_date_hour'].apply(lambda x: 1 if 5 <= x <= 15 else 0)
test['search_date_hour_bin'] = test['search_date_hour'].apply(lambda x: 1 if 5 <= x <= 15 else 0)

In [23]:
not_booked_percentages = not_booked['checkIn_date_dayofweek'].value_counts() / len(not_booked)
not_booked_percentages = [not_booked_percentages[i] for i in range(7)]

is_booked_percentages = is_booked['checkIn_date_dayofweek'].value_counts() / len(is_booked)
is_booked_percentages = [is_booked_percentages[i] for i in range(7)]

trace_not_booked = go.Bar(y = not_booked_percentages, name='Not Booked') # TODO
trace_is_booked = go.Bar(y = is_booked_percentages, name='Booked') # TODO


ticktext = ['دوشنبه', 'سه‌شنبه', 'چهارشنبه', 'پنج‌شنبه', 'جمعه', 'شنبه', 'یکشنبه'] # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Day of Week', tickangle=45, automargin=True,
               tickvals = [0,1,2,3,4,5,6], ticktext= ticktext
 ),
    yaxis=dict(title='Frequency'),
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_day.json')

In [24]:
not_booked_percentages = not_booked['checkIn_date_month'].value_counts() / len(not_booked)
not_booked_percentages = [not_booked_percentages[i] for i in range(1, 13)]

is_booked_percentages = is_booked['checkIn_date_month'].value_counts() / len(is_booked)
is_booked_percentages = [is_booked_percentages[i] for i in range(1, 13)]

trace_not_booked = go.Bar(y = not_booked_percentages, name='Not Booked') # TODO
trace_is_booked = go.Bar(y = is_booked_percentages, name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

layout = go.Layout(
    xaxis=dict(title='Month', tickangle=45, automargin=True,
             ticktext = ticktext ,tickvals = np.arange(0,12)),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./checkIn_date_month.json')

In [25]:
# TODO (Optional): Make season column for checkIn_date_month or search_date_month
def month_to_season(month):
    if month in [12, 1, 2]:
        return 0
    elif month in [3, 4, 5]:
        return 1
    elif month in [6, 7, 8]:
        return 2
    elif month in [9, 10, 11]:
        return 3

train['checkIn_date_season'] = train['checkIn_date_month'].apply(month_to_season)
test['checkIn_date_season'] = test['checkIn_date_month'].apply(month_to_season)

In [26]:
trace_not_booked = go.Scatter(y = not_booked['days_between'].value_counts().sort_values() / len(not_booked), name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_booked['days_between'].value_counts().sort_values() / len(is_booked), name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

In [27]:
trace_not_booked = go.Scatter(y = not_booked['days_between'].value_counts().sort_values() / len(not_booked), name='Not Booked', opacity = 0.5) # TODO
print(not_booked['days_between'].value_counts().sort_values())
trace_is_booked = go.Scatter(y = is_booked['days_between'].value_counts().sort_values() / len(is_booked), name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

In [28]:
trace_not_booked = go.Scatter(y = not_booked['duration'].value_counts().sort_values() / len(not_booked) , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_booked['duration'].value_counts().sort_values() / len(is_booked) , name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Length of Stay', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./los.json')

In [29]:
trace_not_booked = go.Scatter(y = not_booked['duration'].value_counts() / len(not_booked) , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_booked['duration'].value_counts() / len(is_booked) , name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Length of Stay', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./los.json')

In [30]:
trace_not_booked = go.Scatter(y = not_booked['days_between'].value_counts().sort_index() / len(not_booked), name='Not Booked', opacity = 0.5) # TODO
print(not_booked['days_between'].value_counts().sort_values())
trace_is_booked = go.Scatter(y = is_booked['days_between'].value_counts().sort_index() / len(is_booked), name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

In [31]:
trace_not_booked = go.Scatter(y = not_booked['days_between'].value_counts().sort_index() / len(not_booked), name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_booked['days_between'].value_counts().sort_index() / len(is_booked), name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Days between search and checking time', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./days_between.json')

In [32]:
trace_not_booked = go.Scatter(y = not_booked['duration'].value_counts().sort_index() / len(not_booked) , name='Not Booked', opacity = 0.5) # TODO
trace_is_booked = go.Scatter(y = is_booked['duration'].value_counts().sort_index() / len(is_booked) , name='Booked') # TODO

# در ادامه‌ی این قسمت از کد تغییری ایجاد نکنید زیرا نمودار شما توسط سیستم داوری بررسی می‌شود و تغییر در این قسمت ممکن است باعث عدم داوری صحیح شود

data = [trace_is_booked, trace_not_booked]

layout = go.Layout(
    xaxis=dict(title='Length of Stay', tickangle=45, automargin=True),
    yaxis=dict(title='Frequency')
)

fig = go.Figure(data=data, layout=layout)
fig.show()
fig.write_json('./los.json')

In [33]:
# TODO: Add is_abroad column

train['is_abroad'] = np.where(train['user_location_country'] != train['hotel_country'], 1, 0) # TODO
test['is_abroad'] = np.where(test['user_location_country'] != test['hotel_country'], 1, 0) # TODO

In [34]:
# TODO: Preprocessing (Drop Unnecessary Columns) (Optional)
drop_columns = ['user_location_country', 'user_location_region', 'hotel_continent', 'hotel_continent', 'hotel_country', 'is_mobile']

train.drop(columns=drop_columns, inplace=True)
test.drop(columns=drop_columns, inplace=True)
train = train.select_dtypes(include=['number'])
test = test.select_dtypes(include=['number'])

In [35]:
# TODO: Make Validation Set (Optional)

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns=['is_booking']), train['is_booking'], test_size = 0.2, random_state=42) # Optional

In [36]:
# TODO : Design Model
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X_train.shape[1], )))
model.add(keras.layers.Dense(16, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(8, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(4, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.LecunNormal()))

In [37]:
# TODO : Complie the Model
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['AUC', 'accuracy']) # TODO

In [38]:
model.summary()

In [39]:
# TODO : Train your Model

epochs = 50 # TODO
BATCH_SIZE = 1024 # TODO

#checkpoint_cb = None # TODO (Optional)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=BATCH_SIZE, callbacks=[early_stopping_cb]) #TODO

In [40]:
from sklearn.metrics import roc_auc_score

# TODO: Evaluate Model
val_predictions = model.predict(X_valid, batch_size=128)
print(roc_auc_score(y_valid, val_predictions))

In [41]:
# TODO : Design Model
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X_train.shape[1], )))
model.add(keras.layers.Dense(10, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(5, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.LecunNormal()))

In [42]:
# TODO : Complie the Model
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['AUC', 'accuracy']) # TODO

In [43]:
model.summary()

In [44]:
# TODO : Design Model
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X_train.shape[1], )))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(5, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='selu', kernel_initializer=keras.initializers.LecunNormal()))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.LecunNormal()))

In [45]:
# TODO : Complie the Model
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['AUC', 'accuracy']) # TODO

In [46]:
model.summary()

In [47]:
# TODO : Train your Model

epochs = 50 # TODO
BATCH_SIZE = 1024 # TODO

#checkpoint_cb = None # TODO (Optional)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=epochs, batch_size=BATCH_SIZE, callbacks=[early_stopping_cb]) #TODO

In [48]:
from sklearn.metrics import roc_auc_score

# TODO: Evaluate Model
val_predictions = model.predict(X_valid, batch_size=128)
print(roc_auc_score(y_valid, val_predictions))

In [49]:
# Preparing Submission Dataframe

predictions = model.predict(test, batch_size=128)
submission = pd.DataFrame(predictions, columns=['prediction']) # TODO